In [ ]:
import flopy
import os
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import flopy.utils.mflistfile as mflist

In [ ]:
import postprocess

ISWRPQAQ, ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = postprocess.run('June2012', ponds_only= True, riv_only=False, plot_buds=False)

In [ ]:

m = basic.load_model(verbose = True)

In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2012')

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder('June2012')

basic
print(datestart)
print(out_folder)

In [ ]:
def shorten_legend(ax, n=20, hand = None, lab = None):
    
    if hand is None:
        hand, lab = ax.get_legend_handles_labels()
        
    hand, lab = np.array(hand)[np.linspace(0,len(hand)-1,15,dtype = int)], np.array(lab)[np.linspace(0,len(hand)-1,15,dtype = int)]
    
    return hand, lab

In [ ]:
def plot_error(datestart):
    plist = os.path.join(m.model_ws,  'RRlist.lst')


    mf_list = mflist.MfListBudget(plist)
    incremental, cumulative = mf_list.get_budget()
    df_in, df_out = mf_list.get_dataframes(start_datetime=datestart)
    ax = df_out.loc[:,['PERCENT_DISCREPANCY']].plot(grid = True)
    ax.set_title('Cumulative Water Budget Error')
    
    plt.savefig(os.path.join(out_folder, 'bud_cum_err.png'),dpi = 250, bbox_inches = 'tight')
    
    ax = df_in.loc[:,['PERCENT_DISCREPANCY']].plot(grid = True)
    ax.set_title('Instantaneous Water Budget Error')
    plt.savefig(os.path.join(out_folder, 'bud_inst_err.png'),dpi = 250, bbox_inches = 'tight')
    
    ax = df_in.loc[:,['SWR_LEAKAGE_IN']].join(df_out.loc[:,['SWR_LEAKAGE_OUT']]).plot()
    
    ax = df_in.filter(regex = 'STORAGE').plot()
    
    plt.savefig(os.path.join(out_folder, 'bud_leakage.png'),dpi = 250, bbox_inches = 'tight')
    
    plt.figure()
    dftot = df_in.drop(columns = df_in.filter(regex = 'STORAGE|TOTAL').columns).filter(regex = '_IN|_OUT')

    ax = dftot.sum(axis=1).plot()
    ax.set_title('Total Water Balance')
    
    plt.savefig(os.path.join(out_folder, 'bud_basic.png'),dpi = 250, bbox_inches = 'tight')
    
    #w budget
    df_filt = df_in.drop(columns = df_in.filter(regex = 'TOTAL|STORAGE|PERCENT|IN-OUT')).drop(columns = df_in.loc[:,df_in.abs().sum()==0].columns)
    df_filt.filter(regex = '_IN').plot(subplots = True, figsize = (10,10), title = 'GW budget outflows')
    plt.savefig(os.path.join(out_folder, 'bud_gw_basic_inflows.png'),dpi = 250, bbox_inches = 'tight')
    

    ax = df_filt.filter(regex = '_OUT').plot(subplots = True, figsize = (10,10), title = 'GW budget outflows')
    plt.savefig(os.path.join(out_folder, 'bud_gw_basic_outflows.png'),dpi = 250, bbox_inches = 'tight')
    
    return df_in, df_out

df_in, df_out = plot_error(datestart)

In [ ]:
def plot_swr_bud(m, datestart):
    plist = os.path.join(m.model_ws,  'RRlist.lst')
    swrlist = mflist.SwrListBudget(plist,)
    
    incremental , cum = swrlist.get_dataframes(start_datetime=datestart)
    
    incremental.loc[:,cum.sum().abs()>0].resample("1D").sum().filter(regex = 'FLOW').plot(subplots = True,
                                                                                          figsize = (8,8),
                                                                                         title = 'SWR Budget')
    
    plt.savefig(os.path.join(out_folder, 'bud_swr_basic.png'),dpi = 250, bbox_inches = 'tight')
    
    


In [ ]:
def plot_sfr_bud(date_start, seg_filter = None, ):
    plist = os.path.join(m.model_ws,  'Results','sfr_output.cbc')

    sfrbud = flopy.utils.SfrFile(plist)
    
    sfr = sfrbud.df
    sfr.loc[:,'timestep'] = sfr.kstpkper.apply(lambda x:x[0])
    sfr.loc[:,'stressperiod'] = sfr.kstpkper.apply(lambda x:x[1])
    
    sfr.loc[:,'date'] = pd.to_datetime(date_start) + pd.to_timedelta(sfr.loc[:,'stressperiod'], unit = 'D')

    sfrsum = sfr.groupby(['segment', 'date']).sum().filter(regex = 'Q').unstack(['segment'])
    
    if seg_filter is None:
        print('showing all segments')
    else:
        print(f"filtering to {seg_filter}")
        sfrsum = sfrsum.iloc[:, sfrsum.columns.get_level_values(1).isin(seg_filter)]
    
    fig, axes = plt.subplots(2,3, sharex  = True, figsize = (10,10))
    axes = axes.ravel()
    for cnt, col in enumerate(sfrsum.columns.get_level_values(0).unique()):

        sfrsum.loc[:,col].plot(title = col, marker = 'o', ax = axes[cnt])
        if seg_filter is None:
            axes[cnt].legend().remove()
        
    
    plt.tight_layout()
    
    print(f" the SFR flow totals are:\n{sfrsum.stack(1).sum()}")
    
    reaches =  '_'.join([str(x) for x in seg_filter]) if seg_filter is not None else 'all'
    filename = f'sfr_flows_reaches_{reaches}.png'
    print(filename)
    plt.savefig(os.path.join(out_folder, filename),dpi = 250, bbox_inches = 'tight')
    
    return sfrsum, sfr

sfrsum, sfr = plot_sfr_bud(datestart, [51, 56])


In [ ]:
def set_time(df, start_date):
    col = df.filter(regex = 'TIME').columns[0]
    df.loc[:,col] = pd.to_timedelta(df.loc[:,col].astype(int), unit = 's') + pd.to_datetime(start_date)
    
    return df



In [ ]:
def check_len(file):
    file_size = os.path.getsize(file)
    
    return file_size>100.

def SWR(start_date = '1/1/2020'):
    path = os.path.join(m.model_ws, 'Results')
    # files = os.listdir(path)
    # files = [p for p in files if p.endswith('.dat')]

    
    if check_len(os.path.join(path,'ISWRPQM.dat')):
        ISWRPQM = pd.read_csv(os.path.join(path,'ISWRPQM.dat')).rename(columns = lambda x: x.strip())
        print(ISWRPQM.columns)
        
        ISWRPQM = set_time(ISWRPQM, start_date)
        ISWRPQM = ISWRPQM.set_index(['TOTIME','SWRDT','KPER','KSTP','KSWR', 'RCHGRP']) 
    else:
        ISWRPQM = None
    
    if check_len(os.path.join(path,'ISWRPQAQ.dat')):
        ISWRPQAQ = pd.read_csv(os.path.join(path,'ISWRPQAQ.dat'),header = None, skiprows  =1).drop([0])
        c = ['TOTIME','SWRDT','KPER','KSTP','KSWR','REACH','LAYER','GBELEV','STAGE','DEPTH','HEAD','WETPERM','CONDUCT','HEADDIFF','QAQFLOW','na']
        ISWRPQAQ.columns = c
        ISWRPQAQ = ISWRPQAQ.drop(columns = 'na')
        ISWRPQAQ = set_time(ISWRPQAQ, start_date)
        ISWRPQAQ = ISWRPQAQ.set_index(["TOTIME","SWRDT","KPER","KSTP","KSWR","REACH","LAYER"])
    else:
        ISWRPQAQ = None
    
    if check_len(os.path.join(path,'ISWRPRGF.dat')):
        ISWRPRGF = pd.read_csv(os.path.join(path,'ISWRPRGF.dat'),index_col =False).rename(columns = lambda x: x.strip())
        ISWRPRGF = set_time(ISWRPRGF, start_date)
        ISWRPRGF = ISWRPRGF.set_index(['TOTTIME','SWRDT','KPER','KSTP','KSWR', 'RCHGRP'])
        ISWRPRGF = ISWRPRGF.dropna(axis=1, how = 'any')
    else:
        ISWRPRGF = None

    if check_len(os.path.join(path,'ISWRPSTG.dat')):
        ISWRPSTG = pd.read_csv(os.path.join(path,'ISWRPSTG.dat')).rename(columns = lambda x: x.strip())
        ISWRPSTG = set_time(ISWRPSTG, start_date)
        ISWRPSTG = ISWRPSTG.set_index(['TOTIME','SWRDT','KPER','KSTP','KSWR'])   
    else:
        ISWRPSTG = None
    
    if check_len(os.path.join(path,'ISWRPSTR.dat')):
        ISWRPSTR = pd.read_csv(os.path.join(path,'ISWRPSTR.dat')).rename(columns = lambda x: x.strip())
        ISWRPSTR = set_time(ISWRPSTR, start_date)
        ISWRPSTR = ISWRPSTR.set_index(['TOTIME','SWRDT','KPER','KSTP','KSWR', 'REACH']) 
    else:
        ISWRPSTR = None
    
    
    
    return ISWRPQAQ , ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM
    
# ISWRPQAQ, ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = SWR(datestart)
# print('\nISWRPQAQ')

# print('\n\n\nISWRPRGF')

# print('\n\n\nISWRPSTG')

# print('\n\n\nISWRPSTR')

# print('\n\n\nISWRPQM')



In [ ]:
def show_structure():
    if ISWRPSTR is None:
        return
    swrZ_m = ISWRPSTR.groupby(['TOTIME','REACH']).mean()

    for reach in swrZ_m.index.get_level_values("REACH").unique():      
        fz = swrZ_m.loc[swrZ_m.index.get_level_values('REACH')==reach,:].droplevel(1,0)
        fz.plot(subplots = True, figsize = (8,8), title = f"Stream Stages ISWRPSTR\nReach {reach}")
        

        plt.savefig(os.path.join(out_folder,f'SWR_ISWRPSTR_Structures_{reach}.png'), bbox_inches = 'tight')

show_structure()

In [ ]:
def show_stats(ISWRPQAQ, out_folder):
    f = ISWRPQAQ.groupby(['KSTP']).describe().mean().T
    f.to_excel(os.path.join(out_folder,f'ISWRPQAQ_stats.xlsx'))
    f.to_html(os.path.join(out_folder, f'ISWRPQAQ_stats.html'))
    

    
    
def plot_stages():
    if ISWRPQAQ is None:
        return
    fig, ax = plt.subplots(4,2,figsize = (20,20), sharex = True)
    from matplotlib.pyplot import cm

    n = len(ISWRPQAQ.index.get_level_values('REACH').unique())
    color = cm.rainbow(np.linspace(0, 1, n))


    for  reach, c in zip(ISWRPQAQ.index.get_level_values('REACH').unique(), color):
        ISWRPQAQ.query(f"REACH=={reach}").groupby(['TOTIME']).mean().plot(subplots = True, ax =ax.ravel(), color = c, legend = False)

    [(axis.set_title(x),axis.set_facecolor("lightgrey")) for axis,x in zip(ax.ravel(), ISWRPQAQ.columns.tolist())]

    axi = ax[0,1]
    axi.legend()
    handles, lab = axi.get_legend_handles_labels()

    handles, lab = shorten_legend(ax, hand = handles, lab = list(ISWRPQAQ.index.get_level_values('REACH').unique()))

    axi.legend(handles, lab,
               loc = 'upper left', bbox_to_anchor = (1,1))
    plt.savefig(os.path.join(out_folder,f'ISWRPQAQ.png'), bbox_inches = 'tight')
    
def show_stages_indiv():
    if ISWRPSTG is None:
        return
    
    import cycler

    # color = cm.rainbow(np.linspace(0, 1, n))

    if ISWRPSTG.groupby(['TOTIME']).mean().shape[1]>10:
        subplots = False
    else:
        subplots = True

    n = len(ISWRPSTG.columns.unique())
    color = plt.cm.viridis(np.linspace(0, 1,n))

    with  plt.rc_context({"axes.prop_cycle": cycler.cycler('color', color)}):

        ax = ISWRPSTG.groupby(['TOTIME']).mean().plot(subplots = subplots, figsize = (20,20))

    ax.set_facecolor("lightgrey")
    ax.legend(loc = 'upper left', bbox_to_anchor = (1,1))                   

    hand, lab = shorten_legend(ax, n = 20)

    ax.legend(hand, lab, loc = 'upper left', bbox_to_anchor = (1,1))
    plt.suptitle(f"Stream Stages")

    plt.savefig(os.path.join(out_folder,f'SWR_STREAM STAGES.png'), bbox_inches = 'tight')
    

In [ ]:
def plot_swr_reach_all():
    if ISWRPRGF is None:
        return
    
    
    
    fig, ax = plt.subplots(int(np.ceil(ISWRPRGF.shape[1]/2)),2,figsize = (20,20), sharex = True)
    ax = ax.ravel()[:ISWRPRGF.shape[1]]
    from matplotlib.pyplot import cm

    n = len(ISWRPRGF.index.get_level_values('RCHGRP').unique())

    color = cm.rainbow(np.linspace(0, 1, n))


    for  reach, c in zip(ISWRPRGF.index.get_level_values('RCHGRP').unique(), color):
        ISWRPRGF.query(f"RCHGRP=={reach}").groupby(['TOTTIME']).mean().plot(subplots = True, ax =ax.ravel(), color = c, legend = False)



    [(axis.set_title(x),axis.set_facecolor("lightgrey")) for axis,x in zip(ax.ravel(), ISWRPRGF.columns.tolist())]

    axi = ax[1]
    axi.legend()
    hand, lab = axi.get_legend_handles_labels()
    lab = list(ISWRPRGF.index.get_level_values('RCHGRP').unique())
    # axi.legend(handles, ,
    #            loc = 'upper left', bbox_to_anchor = (1,1))

    hand, lab = shorten_legend(axi, n = 40, hand = hand, lab = lab )

    axi.legend(hand, lab,
               loc = 'upper left', bbox_to_anchor = (1,1))

    plt.savefig(os.path.join(out_folder,f'SWR_reach_all.png'), bbox_inches = 'tight')

In [ ]:
def plot_inflow_outflows():
    
    if ISWRPQM is None:
        return
    
    fig, ax = plt.subplots(6,6, sharex = True, figsize = (20,20))

    count = 0
    for reach_time, gdf in ISWRPQM.query("RCHGRP<37 ").groupby(['RCHGRP']):
        g = gdf.set_index('CONNREACH', drop = True, append = True).drop(columns = ['REACHC', 'VLATFLOW']).unstack('CONNREACH').groupby('TOTIME').mean().droplevel(0,1)

        g.plot(ax = ax.ravel()[count])
        ax.ravel()[count].legend(title = None)

        ax.ravel()[count].set_title(f"Reach {reach_time}")
        ax.ravel()[count].set_facecolor("lightgrey")
        count = count+1

    plt.savefig(os.path.join(out_folder,f'reach_flows.png'), bbox_inches = 'tight')
    
    #### next
    fig, ax = plt.subplots(6,8, sharex = True, figsize = (20,20))

    count = 0
    for reach_time, gdf in ISWRPQM.query("RCHGRP>=37 & RCHGRP<=80 ").groupby(['RCHGRP']):
        g = gdf.set_index('CONNREACH', drop = True, append = True).drop(columns = ['REACHC', 'VLATFLOW']).unstack('CONNREACH').groupby('TOTIME').mean().droplevel(0,1)

        g.plot(ax = ax.ravel()[count])
        ax.ravel()[count].legend(title = None)

        ax.ravel()[count].set_title(f"Reach {reach_time}")
        ax.ravel()[count].set_facecolor("lightgrey")
        count = count+1

    plt.savefig(os.path.join(out_folder,f'reach_flows_mid.png'), bbox_inches = 'tight')
    
    fig, ax = plt.subplots(6,6, sharex = True, figsize = (20,20))

    count = 0
    for reach_time, gdf in ISWRPQM.query("RCHGRP>80").groupby(['RCHGRP']):
        g = gdf.set_index('CONNREACH', drop = True, append = True).drop(columns = ['REACHC', 'VLATFLOW']).unstack('CONNREACH').groupby('TOTIME').mean().droplevel(0,1)

        g.plot(ax = ax.ravel()[count])
        ax.ravel()[count].legend(title = None)

        ax.ravel()[count].set_title(f"Reach {reach_time}")
        ax.ravel()[count].set_facecolor("lightgrey")
        count = count+1

    plt.savefig(os.path.join(out_folder,f'reach_flows_bot.png'), bbox_inches = 'tight')

In [ ]:
show_stats(ISWRPQAQ, out_folder)

In [ ]:
ISWRPQAQ, ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = SWR(datestart)

sfrsum, sfr = plot_sfr_bud(datestart)

sfrsum, sfr = plot_sfr_bud(datestart, [51, 56])

plot_stages()

plot_error(datestart)

show_stages_indiv()

show_structure()

plot_swr_reach_all()

plot_inflow_outflows()

show_stats()

In [ ]:
def plot_too_many():
    from matplotlib.pyplot import cm
    print(f"write toomany to proceed with plotting {ISWRPRGF.shape[1]} plots")
    f = input()=='toomany'
    
    if f:
        print('doing plots... hold tight\n')
        print(ISWRPRGF.columns)
        cols = ISWRPRGF.columns
        
        allcols = []
        for c in cols:
            print(f"write 1 if {c} should be plotted")
            ccc = input()=='1'
            if ccc:
                allcols.extend([c])
                
        print(f'will plot {allcols}')
        
        for col in allcols:
            print(f"plotting {col}")
            fig, ax = plt.subplots(11,11, sharex = True, sharey = True, figsize = (20,20))
            count = 0
            fig.suptitle(col)


            # count = 0
            for reach_time, gdf in ISWRPRGF.groupby(['RCHGRP']):
                g = gdf.groupby('TOTTIME').sum().loc[:,col]

                g.plot(ax = ax.ravel()[count])
                ax.ravel()[count].legend(title = None).remove()

                ax.ravel()[count].set_title(f"Reach {reach_time}")
                ax.ravel()[count].set_facecolor("lightgrey")
                # ax.ravel()[count].legend().
                count = count+1
            plt.savefig(os.path.join(f'SWR_Processing/indiv_plots/SWR_reach_{col}.png'), bbox_inches = 'tight')
    else:
        print('not plotting')
            
# plot_too_many()